<span style="color: purple">

Load in stored variables:

</span>

In [ ]:
%store -r data_dir sites_gdf siskiyou_forest_gdf padres_forest_gdf

<span style="color: purple">

Import packages:

</span>

In [ ]:
# Import necessary packages
import os

import matplotlib.pyplot as plt # Overlay pandas and xarray plots
import pandas as pd # Aggregating and data manipulation
import xarray as xr

c:\Users\riede\miniconda3\envs\earth-analytics-python\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## STEP 2c: DATA ACCESS - CLIMATE MODELS

You can use MACAv2 data for historical and future climate data. Be sure
to compare at least two 30-year time periods (e.g. historical vs. 10
years in the future) for at least four of the CMIP models. Overall, you
should be downloading at least 8 climate rasters for each of your sites,
for a total of 16. **You will *need* to use loops and/or functions to do
this cleanly!**.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Write a <strong>function with a numpy-style docstring</strong> that
will download MACAv2 data for a particular climate model, emissions
scenario, spatial domain, and time frame. Then, use loops to download
and organize the 16+ rasters you will need to complete this section. The
<a
href="http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_CMIP5_macav2_catalog2.html">MACAv2
dataset is accessible from their Thredds server</a>. Include an
arrangement of sites, models, emissions scenarios, and time periods that
will help you to answer your scientific question.</p></div></div>

<span style="color: purple">

List of climate models:

</span>

<span style="color: magenta">

* Warm and wet: CanESM2

</span>

<span style="color: limegreen">

* Warm and dry: MIROC-ESM-CHEM

</span>

<span style="color: teal">

* Cold and wet: MRI-CGCM3

</span>

<span style="color: black">

* Cold and dry: GFDL-ESM2M

</span>

<span style="color: purple">

In [ ]:
# Make MACA data directory
maca_dir = os.path.join(data_dir, 'maca-dir')
os.makedirs(maca_dir, exist_ok=True)
maca_dir

maca_pattern = os.path.join(maca_dir, '*.nc')
maca_pattern

In [ ]:
# Define function to convert longitude/latitude
def convert_longitude(longitude):
    """
    Convert longitude range from 0-360 to -180-180
    
    Parameters
    ----------
    longitude : float
        `longitude` values of dataset
    
    Returns
    -------
    longitude : float
        Longitude values converted from 0-360 to -180-180"""
    return (longitude - 360) if longitude > 180 else longitude

In [ ]:
maca_da_list = []
for site_name, site_gdf in {
    'SheyenneGrasslands': sheyenne_grasslands_gdf,
    'CaddoGrasslands': caddo_grasslands_gdf}.items():
    for rcp_value in ['rcp85', 'rcp45']:
        # Define MACA url
        maca_url = (
            'http://thredds.northwestknowledge.net:8080/thredds/dodsC'
            '/MACAV2/CanESM2'
            '/macav2metdata_pr_CanESM2_r1i1p1'
            f'_{rcp_value}'
            '_2076_2080_CONUS'
            '_monthly.nc') 

        # Only download MACA data once
        maca_path = os.path.join(maca_dir, f'maca_{site_name}_{rcp_value}_2076_2080_CONUS_monthly.nc')
        print(maca_path)
        if not os.path.exists(maca_path):
             maca_da = xr.open_dataset(maca_url).squeeze().precipitation
             maca_da.to_netcdf(maca_path)

        # Open & Squeeze MACA dataset
        maca_da = xr.open_dataset(maca_path).squeeze().precipitation

        # Define bounds
        bounds_maca = (site_gdf
                        .to_crs(maca_da.rio.crs)
                        .total_bounds)

        # Change maca_ds longitude values to match the grasslands gdfs
        maca_da = maca_da.assign_coords(
            lon=("lon", [convert_longitude(l) for l in maca_da.lon.values]))

        # Set spatial dimensions of maca_da
        maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')

        # Crop maca_da
        maca_da = maca_da.rio.clip_box(*bounds_maca)
        maca_da_list.append(dict(
            site_name=site_name,
            rcp_value=rcp_value,
            da=maca_da))

maca_df = pd.DataFrame(maca_da_list)

# Display maca_df w/o DataArray column
display(maca_df.drop('da', axis='columns'))

# Pull out the data array for the first row
maca_df.da.values[0]

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Make sure to include a description of the climate data and how you
selected your models. Include a citation of the MACAv2 data</p></div></div>

YOUR CLIMATE DATA DESCRIPTION AND CITATIONS HERE